In [ ]:
import pandas as pd
import geopandas as gpd
import tqdm
from google.cloud import bigquery
from shapely import wkt
from pathlib import Path

In [ ]:
PROJECT_ID = "tm-geodata-warehouse"
DATA_DIR = Path("../../data/01-admin-bounds/")

## GCP Authentication
If necessary, authenticate GCP using this command `gcloud auth login --update-adc`

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID)

## Extract target areas' admin boundaries

In [ ]:
%%time
query = """
SELECT ADM1_EN,
       ADM1_PCODE,
       ADM2_EN,
       ADM2_PCODE,
       ADM3_EN,
       ADM3_PCODE,
       ADM4_EN,
       ADM4_PCODE,
       ST_GEOGFROMTEXT(geometry) AS geometry
FROM  tm-geodata-warehouse.ph_admin_boundary_dec2017.level4_barangay 
WHERE ADM3_EN = 'City of Navotas' OR 
      ADM3_EN = 'City of Mandaluyong' OR
      ADM3_EN = 'City of Muntinlupa' OR 
      ADM3_EN = 'Dagupan City' OR
      ADM3_EN = 'Palayan City' OR
      ADM3_EN = 'Legazpi City' OR
      ADM3_EN = 'Iloilo City' OR
      ADM3_EN = 'Mandaue City' OR
      ADM3_EN = 'Tacloban City' OR
      ADM3_EN = 'Zamboanga City' OR
      ADM3_EN = 'Cagayan de Oro City' OR
      ADM3_EN = 'Davao City'
"""
admin_bound_targets = pd.read_gbq(query, PROJECT_ID, progress_bar_type="tqdm")
admin_bound_targets.head()

In [ ]:
admin_bound_targets["ADM3_EN"].unique()

In [ ]:
admin_bound_targets.groupby("ADM3_EN")["ADM4_EN"].count()

In [ ]:
admin_bound_targets["geometry"] = admin_bound_targets["geometry"].apply(wkt.loads)
gdf = gpd.GeoDataFrame(admin_bound_targets, crs="epsg:4326")

In [ ]:
gdf.to_file(DATA_DIR / "target_admin_bounds.shp")

In [ ]:
admin_bounds_gdf = gpd.read_file(DATA_DIR / "target_admin_bounds.shp")
admin_bounds_gdf = admin_bounds_gdf.rename(
    columns={
        "ADM1_EN": "region_name",
        "ADM1_PCODE": "region_code",
        "ADM2_EN": "province_name",
        "ADM2_PCODE": "province_code",
        "ADM3_EN": "city_name",
        "ADM3_PCODE": "city_code",
        "ADM4_EN": "barangay_name",
        "ADM4_PCODE": "barangay_psgc_code",
    }
)

In [ ]:
admin_bounds_gdf.to_file(DATA_DIR / "renamed_target_admin_bounds.gpkg", driver="GPKG")